In [24]:
import uhd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [25]:

usrp = uhd.usrp.MultiUSRP()


center_freq = 2.44e9# Hz
sample_rate = 56e6 # Hz twice the BW
recordingTime = 1 #s
THRESHOLD = 0.2
# gain = 0# dB

usrp.set_rx_rate(sample_rate, 0)
usrp.set_rx_freq(uhd.libpyuhd.types.tune_request(center_freq), 0)
# usrp.set_rx_gain(gain, 0)
usrp.set_rx_agc(True, 0)

In [26]:
batch = 1000
# Set up the stream and receive buffer
st_args = uhd.usrp.StreamArgs("fc32", "sc16")
st_args.channels = [0]
metadata = uhd.types.RXMetadata()
streamer = usrp.get_rx_stream(st_args)
recv_buffer = np.zeros((1, batch), dtype=np.complex64)

# Start Stream
stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.start_cont)
stream_cmd.stream_now = True
streamer.issue_stream_cmd(stream_cmd)

In [27]:
# Receive Samples
import numpy as np
import time
f = open("RawIQ.iq","wb")

samples = np.zeros(batch, dtype=np.complex64)
start= time.time()
# print(recordingTime <time.time() - start)
while   recordingTime >time.time() - start:
    streamer.recv(recv_buffer, metadata)
    samples = recv_buffer[0]

    if sum(abs(samples))/batch > THRESHOLD:
        samples.tofile(f)
    else:
        np.zeros(batch,dtype=np.complex64).tofile(f)
    

# Stop Stream
stream_cmd = uhd.types.StreamCMD(uhd.types.StreamMode.stop_cont)
streamer.issue_stream_cmd(stream_cmd)

# we will only take the FFT of the first 1024 samples, see text below


DDOOOOOOOOOOOOOOOOOOOOOOOOOO